In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
class Net(nn.Module):
    def __init__(self):
        # The first thing you should always do when you create a module is call its super constructor
        super(Net,self).__init__()
        # 1 input image channel, 6 output channels, 5*5 square convolution
        # kernel
        self.conv1=nn.Conv2d(1,6,5)
        self.conv2=nn.Conv2d(6,16,5)
        # an affine operation: y=Wx+b
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    def forward(self,x):
        # Max pooling over a 2*2 window
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    def num_flat_features(self,x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features=1
        for s in size:
            num_features *= s
        return num_features

    


In [9]:

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [11]:
# learnable parameters
params=list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [12]:
input =torch.randn(1,1,32,32)
out=net(input)
print(out)

tensor([[ 0.1717, -0.0315, -0.1103, -0.0183, -0.0572, -0.0224, -0.0814, -0.0917,
         -0.0876, -0.0447]], grad_fn=<ThAddmmBackward>)


In [13]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [14]:
# loss function
output=net(input)
target=torch.randn(10)
target=target.view(1,-1) # make it the same shape as output
criterion=nn.MSELoss()

loss=criterion(output,target)
print(loss)

tensor(0.7162, grad_fn=<MseLossBackward>)


In [15]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

In [16]:
# backprop
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0051, -0.0013, -0.0157, -0.0115,  0.0017,  0.0033])


In [17]:
# update the weights

# the simplest
# learning_rate=0.01
# for f in net.parameters():
#     f.data.sub_(f.grad.data * learning_rate)

import torch.optim as optim

optimizer=optim.SGD(net.parameters(),lr=0.01)

optimizer.zero_grad() # zero the gradient buffers
output=net(input)
loss=criterion(output,target)
loss.backward()
optimizer.step() # Does the  update